This notebook demonstrates the geodata processing pipeline functions built into `ws3` and `spadesws3`.

Basically, the geodata processing pipeline works in two stages. 

The first stage imports vegetation resource inventory (VRI) vector polygon data and cleans the data (i.e., fix problems with and potentially simplify polygon geometry, discard unwanted attribute data columns, rename remaining attribute data columns, patch bad data values), reproject vectory geometry to target CRS, and exports clean copies of reprojected vector data layers to disk.  

The second stage imports the cleaned vector datasets and compiles multi-band GeoTIFF files that are compatible with input data requirements of the `spades_ws3` SpaDES module (i.e., the harvesting module).

Set a few key variables before sourcing the `spadesws3_params.py` module code. We do not really need to source most of the code in that module, but this way we are certain that the Python environment we are using is almost identical to the environment set up inside the `reticulate` Python bubble in SpaDES when we load the `spades_ws3` module.

In [1]:
!pip uninstall -y ws3

In [3]:
!pip install gurobipy folium
!pip install git+https://github.com/gparadis/ws3@spatial_optimize

  Cloning https://github.com/gparadis/ws3 (to revision spatial_optimize) to /tmp/pip-req-build-4l_kvzp9
  Running command git clone -q https://github.com/gparadis/ws3 /tmp/pip-req-build-4l_kvzp9
  Running command git checkout -b spatial_optimize --track origin/spatial_optimize
  Switched to a new branch 'spatial_optimize'
  Branch 'spatial_optimize' set up to track remote branch 'spatial_optimize' from 'origin'.
  Created wheel for ws3: filename=ws3-0.0.1.post3-py3-none-any.whl size=49850 sha256=34e28e24fdb0acb8b232c9d6eff7ad8ae15a648e2986fd5d07992b3859ae9885
  Stored in directory: /tmp/pip-ephem-wheel-cache-ci21mq_r/wheels/33/57/85/7f53f5ae7b380b5c49fac70d2f2c798c2b093bf3a017b3d73f
Successfully built ws3


In [1]:
dat_path = '../../../input'
basenames = ['bc01']
snk_epsg = 3005 # BC Albers
tolerance = 10.

#prop_names = [u'THLB', u'AU', u'LdSpp', u'Age2015', u'Shape_Area']
#prop_types = [(u'theme0', 'str:10'),
#              (u'theme1', 'str:1'),
#              (u'theme2', 'str:5'), 
#              (u'theme3', 'str:50'), 
#              (u'age', 'int:5'), 
#              (u'area', 'float:10.1')]
update_area_prop = 'area'
pixel_width = 90.
%run -i spadesws3_params

Before running this notebook, you should clone the dev branch of `ws3` and use setuptools "development mode" to deploy symlinks to this cloned repo to `opt/tljh/user/lib/python3.6/site-packages/` by running `sudo -H python setup.py develop` from inside the cloned `ws3` directory, _from a Jupyter terminal_ (TLJH uses its own Python environment, so any deps have to be installed to there not to the system Python environment). 

Compile initial inventory multi-band GeoTIFF files for use with `spades_ws3` SpaDES module.

`rasterize_inventory` is implemented in local module `spadesws3.py`, which basically just calls `ws3.common.rasterize_stands` for reach TSA in `basenames`. 

In [2]:
age_col = 'stAge'

In [4]:
province_codes = ['BC', 'AB', 'SK', 'MB']
mu_codes = {'BC':[1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29 ,30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41],
            'AB':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
            'SK':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
            'MB':[1, 2, 3, 4, 5]}

In [14]:
basenames = ['%s%02d' % (province_code.lower(), mu_code)  for province_code in mu_codes for mu_code in mu_codes[province_code]]
#mu_codes_ = list(zip(*mu_codes_))

In [15]:
basenames

['bc01',
 'bc06',
 'bc07',
 'bc08',
 'bc09',
 'bc10',
 'bc11',
 'bc12',
 'bc13',
 'bc14',
 'bc15',
 'bc16',
 'bc17',
 'bc18',
 'bc19',
 'bc20',
 'bc21',
 'bc22',
 'bc25',
 'bc26',
 'bc27',
 'bc28',
 'bc29',
 'bc30',
 'bc31',
 'bc32',
 'bc34',
 'bc35',
 'bc36',
 'bc37',
 'bc38',
 'bc39',
 'bc40',
 'bc41',
 'ab01',
 'ab02',
 'ab03',
 'ab04',
 'ab05',
 'ab06',
 'ab07',
 'ab08',
 'ab09',
 'ab10',
 'ab11',
 'ab12',
 'ab13',
 'ab14',
 'ab15',
 'ab16',
 'ab17',
 'ab18',
 'ab19',
 'ab20',
 'sk01',
 'sk02',
 'sk03',
 'sk04',
 'sk05',
 'sk06',
 'sk07',
 'sk08',
 'sk09',
 'sk10',
 'sk11',
 'sk12',
 'sk13',
 'sk14',
 'sk15',
 'sk16',
 'sk17',
 'sk18',
 'sk19',
 'mb01',
 'mb02',
 'mb03',
 'mb04',
 'mb05']

In [16]:
import os
for bn in basenames:
    try:
        os.mkdir(tif_path(bn))
    except:
        pass

In [17]:
_ = rasterize_inventory(basenames, shp_path, tif_path, hdt_path, theme_cols, age_col, base_year,
                        age_divisor=1., round_age=10., cap_age=None, d=pixel_width, verbose=True)

#def rasterize_inventory(basenames, shp_path, tif_path, hdt_path, theme_cols, age_col, base_year,
#                        age_divisor=1., round_age=1., cap_age=None, d=100., verbose=True):
#basenames, shp_path, tif_path, hdt_path, theme_cols, age_col, base_year,
#                        age_divisor=1., round_age=1., cap_age=None, d=100., verbose=True

rasterizing ../../../input/gis/shp/bc01.shp
rasterizing ../../../input/gis/shp/bc06.shp
rasterizing ../../../input/gis/shp/bc07.shp
rasterizing ../../../input/gis/shp/bc08.shp
rasterizing ../../../input/gis/shp/bc09.shp
rasterizing ../../../input/gis/shp/bc10.shp
rasterizing ../../../input/gis/shp/bc11.shp
rasterizing ../../../input/gis/shp/bc12.shp
rasterizing ../../../input/gis/shp/bc13.shp
rasterizing ../../../input/gis/shp/bc14.shp
rasterizing ../../../input/gis/shp/bc15.shp
rasterizing ../../../input/gis/shp/bc16.shp
rasterizing ../../../input/gis/shp/bc17.shp
rasterizing ../../../input/gis/shp/bc18.shp
rasterizing ../../../input/gis/shp/bc19.shp
rasterizing ../../../input/gis/shp/bc20.shp
rasterizing ../../../input/gis/shp/bc21.shp
rasterizing ../../../input/gis/shp/bc22.shp
rasterizing ../../../input/gis/shp/bc25.shp
rasterizing ../../../input/gis/shp/bc26.shp
rasterizing ../../../input/gis/shp/bc27.shp
rasterizing ../../../input/gis/shp/bc28.shp
rasterizing ../../../input/gis/s

In [10]:
ls -lah '../../../input/tif/%s' % basenames[0]

total 6.0M
drwxr-xr-x 2 gparadis gparadis  12K Jun 13 20:32 ./
drwxr-xr-x 7 gparadis gparadis 4.0K May  3 22:17 ../
-rw-rw-r-- 1 gparadis gparadis 6.0M Jun 13 20:32 inventory_init.tif
